## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données / Termes de la taxonomie présents dans les données

### **Lire le corpus** 

In [1]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
lng = 'fr'
acteur = '' # Paramètre à indiquer avant de rouler le script

if lng == 'fr':
    file = acteur +'.csv'
if lng == 'en':
    file = acteur + '_en.csv'

In [3]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../01-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 

        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Address', 'Title', 'Type', 'text']]
    return df

def lire_corpus_csv(acteur):
    base_path = '../01-corpus/2-data/1-fr/'
    with open(base_path + acteur + '.csv', encoding='utf-8') as f:
        df = read_csv(f, encoding='utf-8', sep=',')
    return df


In [4]:
data = lire_corpus_csv(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 3568 documents.


In [5]:
base_path = '../01-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Unnamed: 0,Corpus,Sous-corpus,Address,Title,Type,text
0,0,chum,Aide médicale à mourir,https:////www.chumontreal.qc.ca//patients//aid...,Aide médicale à mourir - Je suis patient | CHUM,text//html,Demander l'aide médicale à mourir Faire une de...
1,1,chum,Aide médicale à mourir,https:////www.chumontreal.qc.ca//patients//aid...,Aide médicale à mourir - Je suis un proche | CHUM,text//html,Accompagner un proche qui demande l'aide médic...
2,2,chum,Aide médicale à mourir,https:////www.chumontreal.qc.ca//patients//aid...,Aide médicale à mourir - Don d’organes et don ...,text//html,Don d’organes et don de tissus Le don d’organe...
3,3,chum,Aide médicale à mourir,https:////www.chumontreal.qc.ca//patients//aid...,Aide médicale à mourir - Groupes de soutien | ...,text//html,Groupes de soutien Des groupes qui pourront vo...
4,0,chum,Audiologie,https:////www.chumontreal.qc.ca//patients//aud...,CHUM | Audiologie - Nos services,text//html,Nos services Troubles d'audition Vertiges et d...
...,...,...,...,...,...,...,...
3563,44,chum,Soins et services,https:////www.chumontreal.qc.ca//fiche//preven...,Prévenir les chutes lors de mon rendez-vous à ...,text//html,Prévenir les chutes lors de mon rendez-vous à ...
3564,0,chum,Toxicomanie,https:////www.chumontreal.qc.ca//patients//med...,Soins de santé,text//html,Soins de santé NOS PROGRAMMES Hospitalisation ...
3565,1,chum,Toxicomanie,https:////www.chumontreal.qc.ca//patients//med...,Enseignement,text//html,Enseignement Programme de formation avancée en...
3566,2,chum,Toxicomanie,https:////www.chumontreal.qc.ca//patients//med...,Soutien aux professionnels de la santé,text//html,Soutien aux professionnels de la santé N’hésit...


### **Nettoyage**

In [6]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Au besoin, si on n'arrive pas à traiter la totalité du corpus pour des raisons de performance, on peut prendre un échantillon aléatoire en spécifiant un ratio souhaité.

In [7]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1) # Ratio à modifier au besoin

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 3568 documents


### **Filtrage**
On va filtrer tout de suite filtrer certaines expressions relatives à l'architecture d'information / navigation Web

In [8]:
def filter_mwesw(corpus):
    file_mwesw = '../02-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [9]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [10]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus ; en théorie, le Tree Tagger est capable d'étiquetter
# environ 15 000 tokens / seconde. 
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))
    return tokens

tok(corpus)[:10]

Avec le RegExpTokenizer, notre corpus contient 973306 tokens.
Le POS tagging devrait prendre environ 1 minutes.


['axe',
 'imagerie',
 'et',
 'ingénierie',
 'responsable',
 'de',
 "l'",
 'axe',
 'gilles',
 'soulez']

In [11]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output

In [12]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [13]:
tagged

[['axe', 'VER:pres'],
 ['imagerie', 'NOM'],
 ['et', 'KON'],
 ['ingénierie', 'NOM'],
 ['responsable', 'ADJ'],
 ['de', 'PRP'],
 ["l'", 'DET:ART'],
 ['axe', 'NOM'],
 [':', 'PUN'],
 ['gilles', 'NOM'],
 ['soulez', 'VER:pres'],
 ['STOP', 'NOM'],
 ['m.d.', 'VER:subi'],
 ['STOP', 'NOM'],
 ['m', 'NOM'],
 ['.', 'SENT'],
 ['sc', 'NOM'],
 ['.', 'SENT'],
 ['STOP', 'INT'],
 ['frcpc', 'NOM'],
 ['STOP', 'INT'],
 ['publications', 'NOM'],
 ['des', 'PRP:det'],
 ['chercheurs', 'NOM'],
 ["l'", 'DET:ART'],
 ['axe', 'NOM'],
 ['imagerie', 'NOM'],
 ['et', 'KON'],
 ['ingénierie', 'NOM'],
 ['du', 'PRP:det'],
 ['crchum', 'NOM'],
 ['se', 'PRO:PER'],
 ['donne', 'VER:pres'],
 ['pour', 'PRP'],
 ['mission', 'NOM'],
 ['principale', 'ADJ'],
 ['de', 'PRP'],
 ['mettre', 'VER:infi'],
 ['le', 'DET:ART'],
 ['génie', 'NOM'],
 ['au', 'PRP:det'],
 ['service', 'NOM'],
 ['de', 'PRP'],
 ['la', 'DET:ART'],
 ['santé', 'NOM'],
 ['.', 'SENT'],
 ['il', 'PRO:PER'],
 ['regroupe', 'VER:pres'],
 ['des', 'PRP:det'],
 ['chercheurs', 'NOM'],


### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [14]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=8))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [15]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 7058226 ngrammes.


In [16]:
ngrammes

[(['axe', 'VER:pres'], ['imagerie', 'NOM']),
 (['axe', 'VER:pres'], ['imagerie', 'NOM'], ['et', 'KON']),
 (['axe', 'VER:pres'],
  ['imagerie', 'NOM'],
  ['et', 'KON'],
  ['ingénierie', 'NOM']),
 (['axe', 'VER:pres'],
  ['imagerie', 'NOM'],
  ['et', 'KON'],
  ['ingénierie', 'NOM'],
  ['responsable', 'ADJ']),
 (['axe', 'VER:pres'],
  ['imagerie', 'NOM'],
  ['et', 'KON'],
  ['ingénierie', 'NOM'],
  ['responsable', 'ADJ'],
  ['de', 'PRP']),
 (['axe', 'VER:pres'],
  ['imagerie', 'NOM'],
  ['et', 'KON'],
  ['ingénierie', 'NOM'],
  ['responsable', 'ADJ'],
  ['de', 'PRP'],
  ["l'", 'DET:ART']),
 (['axe', 'VER:pres'],
  ['imagerie', 'NOM'],
  ['et', 'KON'],
  ['ingénierie', 'NOM'],
  ['responsable', 'ADJ'],
  ['de', 'PRP'],
  ["l'", 'DET:ART'],
  ['axe', 'NOM']),
 (['imagerie', 'NOM'], ['et', 'KON']),
 (['imagerie', 'NOM'], ['et', 'KON'], ['ingénierie', 'NOM']),
 (['imagerie', 'NOM'],
  ['et', 'KON'],
  ['ingénierie', 'NOM'],
  ['responsable', 'ADJ']),
 (['imagerie', 'NOM'],
  ['et', 'KON'],
  

### **Extraction des patrons syntaxiques**

In [17]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [18]:
phrases = extract_patterns(ngrammes)

In [19]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=8))

In [20]:
frequencies

FreqDist({('STOP',): 68484, ('de',): 55655, ('.',): 27777, ('la',): 21837, ('et',): 21314, ("l'",): 18482, ('le',): 17328, ('à',): 15197, ('du',): 15041, ('des',): 14624, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [21]:
# Importer l'antidictionnaire pour filtrer les données

# Stopwords fréquents en français (non lemmatisés)
file_path = "../02-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../02-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

In [22]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [23]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [24]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

phrases = filter_len(phrases)

On retire les n-grammes qui apparaissent moins de 5 fois dans le corpus

In [25]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 5]

phrases = filter_freq(phrases)

In [26]:
phrases

[[('imagerie', 'et', 'ingénierie'), ['NOM', 'KON', 'NOM']],
 [('axe', 'imagerie'), ['NOM', 'NOM']],
 [('axe', 'imagerie', 'et', 'ingénierie'), ['NOM', 'NOM', 'KON', 'NOM']],
 [('imagerie', 'et', 'ingénierie'), ['NOM', 'KON', 'NOM']],
 [('axe', 'imagerie'), ['NOM', 'NOM']],
 [('axe', 'imagerie', 'et', 'ingénierie'), ['NOM', 'NOM', 'KON', 'NOM']],
 [('imagerie', 'et', 'ingénierie'), ['NOM', 'KON', 'NOM']],
 [('crchum', 'abrite'), ['NOM', 'VER:pres']],
 [('systèmes', 'biologiques'), ['NOM', 'ADJ']],
 [('développement', 'et', 'validation'), ['NOM', 'KON', 'NOM']],
 [('laboratoire', 'de', 'recherche'), ['NOM', 'PRP', 'NOM']],
 [('laboratoire', 'de', 'recherche', 'en', 'imagerie'),
  ['NOM', 'PRP', 'NOM', 'PRP', 'NOM']],
 [('laboratoire', 'de', 'recherche', 'en', 'imagerie', 'et', 'orthopédie'),
  ['NOM', 'PRP', 'NOM', 'PRP', 'NOM', 'KON', 'NOM']],
 [('recherche', 'en', 'imagerie'), ['NOM', 'PRP', 'NOM']],
 [('recherche', 'en', 'imagerie', 'et', 'orthopédie'),
  ['NOM', 'PRP', 'NOM', 'KON', 

In [27]:
phrases = [[term[0], " ".join(term[1])] for term in phrases]

In [28]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). *Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches*. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [29]:
file_patterns = '../02-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() # Pour prendre les structures syntaxiques attestées dans les MeSH

In [30]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]

In [31]:
terms = filter_patterns(phrases)

In [32]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 32 % des termes
On avait 195538 ngrammes, on en a maintenant 133166.


In [33]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [34]:
terms_patterns

[{'Expression': ('imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM KON NOM',
  'Fréquence': 149},
 {'Expression': ('axe', 'imagerie'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence': 48},
 {'Expression': ('axe', 'imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM NOM KON NOM',
  'Fréquence': 48},
 {'Expression': ('imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM KON NOM',
  'Fréquence': 149},
 {'Expression': ('axe', 'imagerie'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence': 48},
 {'Expression': ('axe', 'imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM NOM KON NOM',
  'Fréquence': 48},
 {'Expression': ('imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM KON NOM',
  'Fréquence': 149},
 {'Expression': ('crchum', 'abrite'),
  'Structure syntaxique': 'NOM VER:pres',
  'Fréquence': 7},
 {'Expression': ('systèmes', 'biologiques'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 7},
 {'Expression': ('développement'

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [35]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [36]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 133166 ngrammes.


In [37]:
frequencies

FreqDist({('STOP',): 68484, ('de',): 55655, ('.',): 27777, ('la',): 21837, ('et',): 21314, ("l'",): 18482, ('le',): 17328, ('à',): 15197, ('du',): 15041, ('des',): 14624, ...})

In [38]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [39]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom

                if p_value < 0.001 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [40]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on ne veut pas appliquer le LLR

In [41]:
significant_coll


[{'Terme': ('imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM KON NOM',
  'Fréquence (TF)': 149,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('axe', 'imagerie'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence (TF)': 48,
  'LLR': 297.8555541377298,
  'p-value': 9.660050488023589e-67},
 {'Terme': ('axe', 'imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM NOM KON NOM',
  'Fréquence (TF)': 48,
  'LLR': 831.5655250775928,
  'p-value': 7.401362611705968e-183},
 {'Terme': ('imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM KON NOM',
  'Fréquence (TF)': 149,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('axe', 'imagerie'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence (TF)': 48,
  'LLR': 297.8555541377298,
  'p-value': 9.660050488023589e-67},
 {'Terme': ('axe', 'imagerie', 'et', 'ingénierie'),
  'Structure syntaxique': 'NOM NOM KON NOM',
  'Fréquence (TF)': 48,
  'LLR': 831.5655250775928,
  'p-value': 7.401362611705968e-183},
 {'Terme': ('imageri

In [42]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 368 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [44]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng

df = df.drop(columns=['p-value'])

### **Filtrage - Fréquence documentaire**

In [45]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [46]:
max_df = round(0.95 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 95% des documents 
min_df = round(0.01 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans moins de 1% des documents
max_df

3390

In [47]:
min_df

36

In [48]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

1280

In [49]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [50]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > min_df} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [51]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [52]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,centre de recherche,1216
1,centre de prélèvement,1280
2,prélèvement guide,1271
3,recherche du chum,1119
4,centre de recherche du chum,1105
...,...,...
454,université de montréal professeur,39
455,tension artérielle,37
456,soins et services,49
457,recherche du chu,1120


In [53]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [54]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF)
0,centre de recherche,NOM PRP NOM,1421,-inf,1216
1,centre de prélèvement,NOM PRP NOM,1304,-inf,1280
2,prélèvement guide,NOM VER:pres,1271,-inf,1271
3,recherche du chum,NOM PRP:det NOM,1199,-inf,1119
4,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1173,-inf,1105
...,...,...,...,...,...
454,université de montréal professeur,NOM PRP ADJ NOM,24,97.634363,39
455,tension artérielle,NOM ADJ,19,309.824178,37
456,soins et services,NOM KON NOM,11,78.225674,49
457,recherche du chu,NOM PRP:det VER:pper,8,51.863171,1120


In [55]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [56]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [57]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [58]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [59]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [60]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [61]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [62]:
df['isMeSHTerm']= False # On set à False puis on va changer pour True si on trouve le terme
df['MeSHID'] = None
df['MesH_prefLabel_fr'] = None
df['MesH_prefLabel_en'] = None

In [63]:
from nltk.tokenize import MWETokenizer
file_path = '../02-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [64]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [65]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = True
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,centre de recherche,NOM PRP NOM,1421,-inf,1216,False,None,None,None
1,centre de prélèvement,NOM PRP NOM,1304,-inf,1280,False,None,None,None
2,prélèvement guide,NOM VER:pres,1271,-inf,1271,False,None,None,None
3,recherche du chum,NOM PRP:det NOM,1199,-inf,1119,False,None,None,None
4,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1173,-inf,1105,False,None,None,None
...,...,...,...,...,...,...,...,...,...
454,université de montréal professeur,NOM PRP ADJ NOM,24,97.634363,39,False,None,None,None
455,tension artérielle,NOM ADJ,19,309.824178,37,True,None,None,None
456,soins et services,NOM KON NOM,11,78.225674,49,False,None,None,None
457,recherche du chu,NOM PRP:det VER:pper,8,51.863171,1120,False,None,None,None


In [66]:
# Termes MeSH présents dans notre corpus 
df[df['isMeSHTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
22,soins intensifs,NOM ADJ,453,-inf,161,True,None,None,None
35,fibrillation auriculaire,NOM ADJ,293,-inf,43,True,None,None,None
46,reconstruction mammaire,NOM ADJ,202,-inf,50,True,None,None,None
62,produits naturels,NOM ADJ,168,-inf,84,True,None,None,None
63,boissons énergisantes,NOM ADJ,167,-inf,86,True,None,None,None
66,syndromes parkinsoniens,NOM ADJ,165,-inf,59,True,None,None,None
67,maladie de parkinson,NOM PRP NOM,163,-inf,62,True,None,None,None
96,santé publique,NOM ADJ,145,1572.986426,95,True,None,None,None
99,santé mentale,NOM ADJ,144,1579.079685,68,True,None,None,None
101,médecine nucléaire,NOM ADJ,141,-inf,119,True,None,None,None


### **Extraction de termes existant dans la taxonomie**

In [67]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme

In [68]:
file_path = '../02-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    taxo_terms = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

taxo_terms

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [69]:
for t in df['Terme'].tolist():
    if t in taxo_terms:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = True
df[df['isTaxoTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm
1,centre de prélèvement,NOM PRP NOM,1304,-inf,1280,False,None,None,None,True
22,soins intensifs,NOM ADJ,453,-inf,161,True,None,None,None,True
34,recherche clinique,NOM ADJ,294,1106.288321,183,False,None,None,None,True
35,fibrillation auriculaire,NOM ADJ,293,-inf,43,True,None,None,None,True
42,centre hospitalier,NOM ADJ,215,-inf,147,False,None,None,None,True
46,reconstruction mammaire,NOM ADJ,202,-inf,50,True,None,None,None,True
67,maladie de parkinson,NOM PRP NOM,163,-inf,62,True,None,None,None,True
96,santé publique,NOM ADJ,145,1572.986426,95,True,None,None,None,True
97,médecin de famille,NOM PRP NOM,145,-inf,100,False,None,None,None,True
99,santé mentale,NOM ADJ,144,1579.079685,68,True,None,None,None,True


### **Mapping to MeSH IDs**

Lire le fichier XML contenant le MeSH Bilingue

Le MeSH 2019 comprend 29 351 descripteurs  
https://mesh.inserm.fr/FrenchMesh/presentation.htm

In [70]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
base_path = '../02-filtrage/MeSH/'

tree = ET.parse(base_path + 'fredesc2019.xml')
root = tree.getroot()

def flatten(l):
    return [item for sublist in l for item in sublist]

data_mesh = [{'mesh_id' : x.find('DescriptorUI').text.strip('\n'), \
         'label_fr' : x.find('DescriptorName').find('String').text.split('[')[0], \
         'label_en' : x.find('DescriptorName').find('String').text.split('[')[1].strip(']'), \
         'synonymes (en/fr)' : flatten([[term.find('String').text for term in concept.find('TermList').findall('Term')] for concept in x.find('ConceptList').findall('Concept')]) \
         } for x in root.findall('DescriptorRecord')]

In [71]:
for t in df[df['isMeSHTerm'] == True]['Terme'].tolist():
    for d in data_mesh:
        if t in [str(x).lower() for x in d['synonymes (en/fr)']]:
            df.loc[df['Terme'] == t, 'MeSHID'] = d['mesh_id']
            df.loc[df['Terme'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
            df.loc[df['Terme'] == t, 'MesH_prefLabel_en'] = d['label_en']

df = df[['Terme', 'Structure syntaxique',	'Fréquence (TF)', 'Fréquence documentaire (DF)', 'LLR', 'isMeSHTerm', 'isTaxoTerm', 'MeSHID', 'MesH_prefLabel_fr', 'MesH_prefLabel_en']]

In [72]:
df.insert(0, 'Corpus', acteur)
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,chum,centre de recherche,NOM PRP NOM,1421,1216,-inf,False,False,None,None,None
1,chum,centre de prélèvement,NOM PRP NOM,1304,1280,-inf,False,True,None,None,None
2,chum,prélèvement guide,NOM VER:pres,1271,1271,-inf,False,False,None,None,None
3,chum,recherche du chum,NOM PRP:det NOM,1199,1119,-inf,False,False,None,None,None
4,chum,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1173,1105,-inf,False,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
454,chum,université de montréal professeur,NOM PRP ADJ NOM,24,39,97.634363,False,False,None,None,None
455,chum,tension artérielle,NOM ADJ,19,37,309.824178,True,True,D062186,Pression artérielle,Arterial Pressure
456,chum,soins et services,NOM KON NOM,11,49,78.225674,False,False,None,None,None
457,chum,recherche du chu,NOM PRP:det VER:pper,8,1120,51.863171,False,False,None,None,None


In [73]:
output_path = path.join('../02-filtrage/output/', acteur + '_candidate-terms.csv') 
df.to_csv(output_path)